# Machine Learning to predict number of customers

In [36]:
import pandas as pd
import numpy as np

df = pd.read_csv("training_data_customers.csv")
final_df = df.drop(columns="Unnamed: 1")

In [37]:
from sklearn.ensemble import RandomForestRegressor

## Random Forest
# Best parameters found
best_params = {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Create and train the Random Forest model with the best parameters
rf_model = RandomForestRegressor(random_state=42, **best_params)  # **best_params unpacks the dictionary

In [38]:
#Update Customer Data
def update_customer_data(new_customer_counts):
    """Updates customer data for shifts 1, 2, and 3 with rounded values.

    Args:
        new_customer_counts: A NumPy array or a list/tuple that can be converted
                             to a NumPy array of customer counts.

    Returns:
        The updated DataFrame.
    """

    global final_df

    new_customer_counts = np.array(new_customer_counts) # Ensure it's a NumPy array
    if len(new_customer_counts) != 3:
        raise ValueError("new_customer_counts must be a list or tuple of length 3.")


    rounded_counts = np.round(new_customer_counts).astype(int) # Round and convert to integers

    last_group = final_df['Group'].max()
    if pd.isna(last_group):
        new_group = 1
    else:
        new_group = last_group + 1

    new_rows = []
    for i, customers in enumerate(rounded_counts):
        new_rows.append({'Group': new_group, 'Shift': i + 1, 'Customers': customers})

    final_df = pd.concat([final_df, pd.DataFrame(new_rows)], ignore_index=True)
    return final_df

In [39]:
#Retraining the model
def retraining(final_df):
    """Retrains the model with the updated DataFrame.

    Args:
        final_df: The updated pandas DataFrame.

    Returns:
        The retrained model (it's generally better to return the model itself).
    """
    X = final_df[['Shift']]
    y = final_df['Customers']

    rf_model.fit(X, y)  # Retrain the model

    return rf_model

In [43]:
# Retrain the model:
retrained_model = retraining(final_df)  # Store the retrained model

#Now you can use predict then retrained the model:
customer_data = np.array([[1], [2], [3]])
new_predictions = retrained_model.predict(customer_data)
final_df = update_customer_data(new_predictions)
# Access all elements
all_elements = new_predictions.flatten()
print(all_elements)

[47.66881115 60.08939703 30.32488925]


C:\Users\yuliastu\Music\Feni\Uni\Master Dokumen\Intelligent Agent\mesa-restaurant-agents\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [44]:
# Retrain the model:
retrained_model = retraining(final_df)  # Store the retrained model

#Now you can use predict then retrained the model:
customer_data = np.array([[1], [2], [3]])
new_predictions = retrained_model.predict(customer_data)
final_df = update_customer_data(new_predictions)
# Access all elements
all_elements = new_predictions.flatten()
print(all_elements)

[47.66927773 60.08890549 30.32501606]


C:\Users\yuliastu\Music\Feni\Uni\Master Dokumen\Intelligent Agent\mesa-restaurant-agents\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [45]:
num_rows = final_df.shape[0]
print(f"The number of rows in final_df is: {num_rows}")

The number of rows in final_df is: 3012


In [46]:
final_df = update_customer_data([50,30,25])
num_rows = final_df.shape[0]
print(f"The number of rows in final_df is: {num_rows}")

The number of rows in final_df is: 3015


In [47]:
# Customer demands
customer_demands = {1: 63, 2: 90, 3: 45}

# Create a new dictionary with the same keys as customer_demands and values from new_predictions
predicted_demands = {key: new_predictions[i] for i, key in enumerate(customer_demands.keys())}

print(predicted_demands)

{1: np.float64(47.66927773454449), 2: np.float64(60.08890548583444), 3: np.float64(30.325016060610015)}


In [1]:
import mesa
print(mesa.__version__)

3.1.0


# USE LP to Solve Optimization

In [20]:
import mesa
import random
import pyoptinterface as poi
from pyoptinterface import highs

## Constraints and Goal
- Full-time waiters can work at most 2 shifts per day, without the requirement for continuous shifts.
- Part-time waiters can work at most 1 shift per day.
- The total capacity in each shift meets or exceeds customer demands.
- Each shift has at least 2 waiters.
- Only specific waiters can work in each shift based on the waiters_per_shift list.
- Group A und Group B has Relationship

Objective minimize number of waiters. Only allocate what i needed based on constraints

## Handle No enough Staff Problem in higher demand
When the demand is high and there aren't enough waiters to meet all the constraints:

1. Relax Constraints
Adjust Shift Requirements: Allow part-time waiters to work more than one shift per day or allow full-time waiters to work more than two shifts.


2. Prioritize Constraints (Hate Group and special request for specific Shifts will be relaxed)
Relax Less Critical Constraints: Identify which constraints are less critical and relax them to find a feasible solution.
Prioritize Critical Shifts: Ensure that the most critical shifts are fully staffed, even if it means understaffing less critical shifts.

In [48]:
#VARIABLE

#Waiter Type
waiter_types = ["Fulltime", "Parttime"]
waite_total = {"Fulltime": 5, "Parttime": 4}

# Waiter Name
waiter_name = {
    "Fulltime": ["Ana", "Bob", "Alice", "Putri", "Lala"],
    "Parttime": ["Laura", "Bill", "Adi", "Jon"]
}

shifts = [1, 2, 3]
customer_demands = predicted_demands
capacity_waiter = 20

# Initialize the model and decision variables
model = highs.Model()

# Define variables for each waiter in each shift
waiter_vars = {}
for waiter_type in waiter_types:
    for waiter in waiter_name[waiter_type]:
        for shift in shifts:
            var_name = f"{waiter}_{shift}"
            waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)

# List of waiters that can work in each shift
waiters_per_shift = {
    1: ["Ana", "Bob", "Alice"],
    2: ["Ana", "Bob", "Laura", "Bill", "Adi", "Jon"],
    3: ["Alice", "Putri", "Lala", "Laura", "Bill", "Adi", "Jon"]
}

# Define Group of people who prefer not working with people in another group 
group_A = ["Ana", "Bob", "Alice"]
group_B = ["Putri", "Lala", "Laura"]

def solve_scheduling_problem(waiter_vars, relax_constraints=False):
    model = highs.Model()  # Create a new instance of the model

    # Re-define variables for each waiter in each shift
    for waiter_type in waiter_types:
        for waiter in waiter_name[waiter_type]:
            for shift in shifts:
                var_name = f"{waiter}_{shift}"
                waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)

    # Constraint 1: Each full-time waiter can work at most 2 shifts per day
    for waiter in waiter_name["Fulltime"]:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            2 if not relax_constraints else 3,  # Relax constraint if needed
            name=f"{waiter}_fulltime_max_two_shifts"
        )

    # Constraint 2: Each part-time waiter can work at most 1 shift per day
    for waiter in waiter_name["Parttime"]:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            1 if not relax_constraints else 2,  # Relax constraint if needed
            name=f"{waiter}_parttime_max_one_shift"
        )

    # Constraint 3: The total capacity in each shift must meet or exceed customer demands
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] * capacity_waiter for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
            poi.Geq,
            customer_demands[shift],
            name=f"shift_{shift}_demand"
        )

    # Constraint 4: Each shift must have at least 2 waiters
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
            poi.Geq,
            2,
            name=f"shift_{shift}_min_two_waiters"
        )

    # Constraint 5: Only specific waiters can work in each shift
    if not relax_constraints:  # Relax Constraint 2, less critical prioritization
        for shift in shifts:
            for waiter_type in waiter_types:
                for waiter in waiter_name[waiter_type]:
                    if waiter not in waiters_per_shift[shift]:
                        model.add_linear_constraint(
                            waiter_vars[f"{waiter}_{shift}"],
                            poi.Eq,
                            0,
                            name=f"{waiter}_not_in_shift_{shift}"
                        )

    # Constraint 6: People in Group A and Group B cannot work together in the same shift
    if not relax_constraints:  # Relax Constraint 2, less critical prioritization
        for shift in shifts:
            for waiter_A in group_A:
                for waiter_B in group_B:
                    model.add_linear_constraint(
                        waiter_vars[f"{waiter_A}_{shift}"] + waiter_vars[f"{waiter_B}_{shift}"],
                        poi.Leq,
                        1,
                        name=f"group_A_B_not_together_shift_{shift}"
                    )

    # Objective: Minimize the total number of waiters assigned
    model.set_objective(
        poi.quicksum(waiter_vars[var] for var in waiter_vars),
        poi.ObjectiveSense.Minimize
    )

    model.set_model_attribute(poi.ModelAttribute.Silent, False)
    model.optimize()

    return model


# First attempt with original constraints
model = solve_scheduling_problem(waiter_vars, relax_constraints=False)

# TerminationStatusCode.OPTIMAL
termination_status = model.get_model_attribute(poi.ModelAttribute.TerminationStatus)
print(f"Initial attempt termination status: {termination_status}")
# Variable to track if relaxed constraints were used
used_relaxed_constraints = False

# Check if a solution is found
if termination_status != poi.TerminationStatusCode.OPTIMAL:
    print("No optimal solution found, trying with relaxed constraints...")
    # If no solution found, relax constraints and try again
    model = solve_scheduling_problem(waiter_vars, relax_constraints=True)
    termination_status = model.get_model_attribute(poi.ModelAttribute.TerminationStatus)
    used_relaxed_constraints = True

# Print the schedule if a solution is found
if termination_status == poi.TerminationStatusCode.OPTIMAL:
    schedule = {shift: [] for shift in shifts}
    for var in waiter_vars:
        if model.get_value(waiter_vars[var]) > 0.5:  # If the waiter is assigned to the shift
            waiter, shift = var.rsplit('_', 1)
            schedule[int(shift)].append(waiter)

    for shift in shifts:
        print(f"Shift {shift}: {', '.join(schedule[shift])}")
    
    if used_relaxed_constraints:
        print("The solution was found using relaxed constraints.")
    else:
        print("The solution was found using original constraints.")
else:
    print("No feasible solution found, even with relaxed constraints.")

Initial attempt termination status: TerminationStatusCode.OPTIMAL
Shift 1: Ana, Bob, Alice
Shift 2: Ana, Bob, Bill, Jon
Shift 3: Laura, Adi
The solution was found using original constraints.
